In [1]:
'''
将原始小时数据重采样为每天数据或每月数据
'''
import pandas as pd
import numpy as np

In [27]:
DATA_DIR = '../../../DATA.csv'

In [28]:
df  = pd.read_csv(DATA_DIR)

In [13]:
df.head()

,Source.Name,date,hour,type,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台
0,china_cities_20220402.csv,20220402.0,0.0,AQI,51.0,56.0,75.0,52.0,52.0,79.0,72.0,26.0,50.0,56.0,47.0,62.0,84.0
1,china_cities_20220402.csv,20220402.0,0.0,PM2.5,25.0,30.0,52.0,29.0,26.0,58.0,44.0,9.0,29.0,34.0,28.0,44.0,62.0
2,china_cities_20220402.csv,20220402.0,0.0,PM2.5_24h,14.0,31.0,29.0,31.0,17.0,37.0,25.0,12.0,16.0,19.0,30.0,28.0,32.0
3,china_cities_20220402.csv,20220402.0,0.0,PM10,52.0,62.0,100.0,53.0,54.0,85.0,93.0,21.0,50.0,61.0,47.0,73.0,98.0
4,china_cities_20220402.csv,20220402.0,0.0,PM10_24h,32.0,55.0,65.0,51.0,40.0,65.0,66.0,26.0,35.0,36.0,45.0,51.0,61.0


In [29]:
df['tag']  = df.type.apply(lambda x: True if x in ['O3_24h','NO2_24h','SO2_24h','PM2.5_24h'] else False)

In [30]:
df = df[df.tag == True]

In [31]:
groups = df.groupby(['type','hour'])

In [32]:
dates = df.date.unique()
types = df.type.unique()
col_names = list(df.columns)

In [33]:
# 获取某一指标某天的数据
groups.get_group((types[0],23.0))

,Source.Name,date,hour,type,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台,tag
4,china_cities_20150102.csv,20150102.0,23.0,O3_24h,50.0,52.0,57.0,61.0,60.0,45.0,67.0,64.0,76.0,44.0,60.0,66.0,46.0,True
349,china_cities_20150103.csv,20150103.0,23.0,O3_24h,23.0,22.0,13.0,17.0,16.0,31.0,53.0,45.0,37.0,11.0,39.0,43.0,17.0,True
709,china_cities_20150104.csv,20150104.0,23.0,O3_24h,30.0,13.0,7.0,33.0,39.0,22.0,35.0,47.0,29.0,18.0,26.0,54.0,12.0,True
889,china_cities_20150105.csv,20150105.0,23.0,O3_24h,71.0,49.0,36.0,63.0,35.0,15.0,51.0,68.0,86.0,43.0,39.0,59.0,23.0,True
1219,china_cities_20150106.csv,20150106.0,23.0,O3_24h,64.0,56.0,129.0,72.0,43.0,52.0,62.0,81.0,82.0,43.0,62.0,62.0,40.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924457,china_cities_20220329.csv,20220329.0,23.0,O3_24h,81.0,91.0,69.0,91.0,108.0,111.0,73.0,109.0,104.0,95.0,91.0,102.0,87.0,True
924817,china_cities_20220330.csv,20220330.0,23.0,O3_24h,61.0,95.0,61.0,94.0,105.0,74.0,61.0,90.0,84.0,90.0,106.0,102.0,52.0,True
925177,china_cities_20220331.csv,20220331.0,23.0,O3_24h,92.0,99.0,108.0,101.0,108.0,103.0,104.0,116.0,98.0,102.0,100.0,112.0,105.0,True
925537,china_cities_20220401.csv,20220401.0,23.0,O3_24h,92.0,123.0,114.0,113.0,112.0,130.0,121.0,104.0,106.0,133.0,117.0,120.0,123.0,True


In [34]:
data = {type_:groups.get_group((type_,23.0)) for type_ in types}

In [35]:
for type_,mdata in data.items():    
    mdata['date'] = pd.to_datetime(mdata['date'].apply(lambda x: str(int(x))))

/var/folders/sb/pk_jx04n7wx0bf41lw3ktd240000gn/T/ipykernel_6638/2376657553.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdata['date'] = pd.to_datetime(mdata['date'].apply(lambda x: str(int(x))))


In [36]:
month_data = {type_:mdata.resample('M',on='date').mean() for type_,mdata in data.items()}
month_data['O3_24h']

,hour,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台,tag
date,,,,,,,,,,,,,,,
2015-01-31,23.0,48.233333,44.633333,46.466667,52.500000,41.400000,48.800000,47.033333,63.566667,61.666667,36.100000,47.766667,61.633333,35.700000,1.0
2015-02-28,23.0,69.444444,60.555556,63.444444,78.000000,61.074074,73.074074,72.555556,83.259259,85.703704,56.370370,76.518519,84.888889,61.259259,1.0
2015-03-31,23.0,99.857143,69.379310,107.379310,108.206897,83.172414,83.655172,101.586207,118.137931,118.103448,82.724138,104.206897,128.241379,91.517241,1.0
2015-04-30,23.0,141.642857,89.321429,133.142857,153.464286,110.000000,115.571429,140.500000,127.928571,138.250000,126.571429,135.464286,160.214286,114.428571,1.0
2015-05-31,23.0,165.384615,139.807692,138.961538,171.961538,95.423077,115.730769,167.423077,143.115385,172.269231,150.923077,163.769231,167.153846,128.615385,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,23.0,49.129032,49.096774,51.677419,51.838710,59.096774,60.935484,48.709677,70.354839,51.645161,46.225806,54.129032,53.225806,54.032258,1.0
2022-01-31,23.0,52.483871,59.645161,49.290323,60.709677,62.129032,53.451613,52.096774,71.645161,59.580645,56.290323,62.774194,60.161290,53.129032,1.0
2022-02-28,23.0,77.214286,85.428571,85.392857,84.107143,85.857143,91.035714,85.178571,90.321429,85.285714,81.571429,87.678571,91.285714,87.964286,1.0


In [22]:
year_data = {type_:mdata.resample('Y',on='date').mean() for type_,mdata in data.items()}

In [41]:
year_data['O3_24h']

,hour,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台,tag
date,,,,,,,,,,,,,,,
2015-12-31,23.0,115.839650,91.541787,98.706052,113.876081,75.345821,95.717579,115.766571,109.766571,117.714697,104.306358,107.155620,120.230548,94.847262,1.0
2016-12-31,23.0,112.591160,100.340659,103.766484,113.607143,97.480769,108.785714,107.401099,115.620879,116.519231,110.774725,118.831956,125.741758,102.365385,1.0
2017-12-31,23.0,119.285714,124.505587,121.734637,127.432961,118.717877,123.905028,132.229050,123.019553,114.067039,128.122905,129.751397,121.846369,131.047486,1.0
2018-12-31,23.0,118.054131,126.508621,126.667622,123.468571,109.764368,131.730659,133.725714,121.846821,114.905714,117.977011,131.957020,127.997126,131.477143,1.0
2019-12-31,23.0,113.289017,124.209632,122.940510,117.356941,111.252125,128.575071,122.643059,113.257790,106.838068,119.971671,119.325779,121.317280,124.792614,1.0
2020-12-31,23.0,109.443836,117.772603,115.249315,116.682192,111.906849,120.663934,112.939891,109.608219,107.019126,117.082192,122.240437,123.172603,120.758904,1.0
2021-12-31,23.0,99.741047,107.988981,114.842975,107.812672,107.300275,117.848485,111.341598,107.247934,97.399449,109.330579,113.906336,116.655647,119.600551,1.0
2022-12-31,23.0,73.076087,80.989130,77.326087,81.836957,83.445652,81.282609,79.478261,88.728261,80.293478,79.086957,85.391304,86.347826,79.771739,1.0


In [37]:
year_data.keys()

dict_keys(['PM2.5_24h', 'SO2_24h', 'NO2_24h', 'O3_24h'])

In [50]:
year_data['SO2_24h']

,hour,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台,tag
date,,,,,,,,,,,,,,,
2015-12-31,23.0,13.134111,29.648415,47.783862,48.907781,38.100865,45.507205,55.170029,34.296830,22.939481,23.257225,39.763689,36.000000,61.023055,1.0
2016-12-31,23.0,10.052486,20.453297,39.892857,46.329670,27.637363,42.079670,39.126374,22.005495,17.346154,17.741758,36.055096,30.409341,52.607143,1.0
2017-12-31,23.0,7.700280,16.240223,33.005587,40.882682,25.678771,36.592179,28.745810,14.773743,16.994413,13.678771,30.888268,18.731844,38.821229,1.0
2018-12-31,23.0,6.071225,11.709770,22.000000,33.877143,20.020115,21.260745,20.645714,13.861272,13.348571,10.816092,23.510029,13.965517,25.451429,1.0
2019-12-31,23.0,4.511561,11.005666,16.509915,23.209632,19.033994,15.733711,14.586402,10.991501,13.670455,8.569405,17.665722,12.725212,19.786932,1.0
2020-12-31,23.0,3.668493,8.594521,12.405479,18.202740,14.712329,14.631148,11.180328,11.372603,12.174863,8.197260,11.071038,12.350685,14.780822,1.0
2021-12-31,23.0,2.953168,8.570248,9.223140,9.994490,11.473829,12.077135,7.870523,8.743802,10.914601,7.674931,8.173554,12.041322,9.834711,1.0
2022-12-31,23.0,2.815217,8.760870,9.097826,9.021739,9.293478,9.326087,8.652174,5.402174,8.054348,5.793478,9.304348,10.956522,7.619565,1.0


# 指数平滑模型

In [38]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [107]:
def SES(df,label):
    dt=df.loc[:,label]
    fit1 = SimpleExpSmoothing(dt).fit()
    return fit1.forecast(1).values,fit1.sse

#HOLT线性趋势模型
def holtmethod(df,label):
    dt = df.loc[:,label]
    fit1 = Holt(dt).fit(smoothing_level=0.7, smoothing_trend=0.2, optimized=False)
    return fit1.forecast(1).values

#HOLT指数趋势模型

def holt2method(df,label):
    dt = df.loc[:,label]
    fit1 = Holt(dt, exponential=True).fit(smoothing_level=0.7, smoothing_trend=0.2, optimized=False)
    return fit1.forecast(1).values

#阻尼趋势模型
def holt3method(df,label):
    dt = df.loc[:,label]
    fit1 = Holt(dt, exponential=False, damped_trend=True).fit(smoothing_level=0.7, smoothing_trend=0.2, damping_trend=0.8, optimized=False)
    return fit1.forecast(1).values

#乘法阻尼趋势
def holt4method(df,label):
    dt = df.loc[:,label]
    fit1 = Holt(dt, exponential=True, damped_trend=True).fit(smoothing_level=0.7, smoothing_trend=0.2, damping_trend=0.8, optimized=False)
    return fit1.forecast(1).values

#三次指数平滑 Holt winters 加法模型

def holt5method(df,label):
    dt = df.loc[:,label]
    fit1 = ExponentialSmoothing(dt, trend='add').fit(smoothing_level=0.7, smoothing_trend=0.2, smoothing_seasonal=0.1, optimized=False)
    return fit1.forecast(1).values   
#h-w 乘法模型
def holt6method(df,label):
    dt = df.loc[:,label]
    fit1 = ExponentialSmoothing(dt, trend='add').fit(smoothing_level=0.7, smoothing_trend=0.2, smoothing_seasonal=0.1, optimized=False)
    return fit1.forecast(1).values 

#hw-damped 
def holt7method(df,label):
    dt = df.loc[:,label]
    fit1 = ExponentialSmoothing(dt , trend='add').fit(smoothing_level=0.7, smoothing_trend=0.2, smoothing_seasonal=0.1, optimized=False)
    return fit1.forecast(1).values 
    


In [108]:
def SmoothMethod(data,key):
    sesforcast = []
    sessse = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres , modelsse= SES(data,label)
        sesforcast.extend(forecastres)
        sessse.append(modelsse)
    print(sesforcast)
    print(sessse)

    holt1forecast = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres = holtmethod(data,label)
        holt1forecast.extend(forecastres)
    print(holt1forecast)

    holt2forecast = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres = holt2method(data,label)
        holt2forecast.extend(forecastres)
    print(holt2forecast)

    holt3forecast = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres = holt3method(data,label)
        holt3forecast.extend(forecastres)
    print(holt3forecast)

    holt4forecast = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres = holt4method(data,label)
        holt4forecast.extend(forecastres)
    print(holt4forecast)

    holt5forecast = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres = holt5method(data,label)
        holt5forecast.extend(forecastres)
    print(holt5forecast)

    holt6forecast = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres = holt6method(data,label)
        holt6forecast.extend(forecastres)
    print(holt6forecast)

    holt7forecast = []
    for label in ['北京', '天津', '石家庄', '唐山', '秦皇岛', '邯郸', '保定', '张家口', '承德', '廊坊','沧州', '衡水', '邢台']:
        forecastres = holt7method(data,label)
        holt7forecast.extend(forecastres)
    print(holt7forecast)

    

    

In [109]:
SmoothMethod(year_data['SO2_24h'],'SO2_24h')

/usr/local/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(
/usr/local/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:132: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  date_key = Timestamp(key, freq=base_index.freq)


[2.8152173933599727, 8.76086956237099, 9.097826088833646, 9.021739144929908, 9.293478293428974, 9.326086997515548, 8.65217390139598, 5.402173962837613, 8.054347868708145, 5.7934782889160035, 9.30434780923681, 10.956521755297214, 7.619565250399547]
[21.357140629708407, 129.17150558133378, 287.95580050260287, 292.5802843952019, 180.23238020768812, 322.7684186622343, 490.64892122038134, 230.66565486406049, 56.81090665263656, 64.12196260451087, 182.2080807299359, 193.2884048680028, 526.149115776034]
[0.7371042392512359, 3.803448799337703, 1.8483856942852963, 2.870818394069377, 3.617287715164829, 4.1726780725913875, -1.5391940633054606, -0.058502112284639196, 5.632264951393365, 2.7488432401631484, 3.5699144901148343, 6.758026432074187, -1.0556536024282064]
[2.1720340096458286, 6.898206388315198, 6.955783937139576, 7.431371466372959, 7.581467341989667, 7.927957428169259, 6.138843263694356, 4.535651017290495, 7.233250505201338, 4.912107721066204, 7.202229992317945, 9.340064521769431, 5.970923